In [1]:
# -*-coding:utf-8-*-
import torch
from torch.autograd import grad
import os
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import csv
import gc
import yaml

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, random_split, Subset, ConcatDataset

torch.autograd.set_detect_anomaly(True)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
config_dict = {
    'exp-base-dir':'exps',
    'y_target':'delta_e',
    # Number of covariates
    'dim_x': None,
    #func r
    'alpha_for_r':10,
    #func attack
    'attack_lr': 7e-2,
    #func train_theta
    'opt_lr':0.01,
    'advadv_rate':5,
    'grad_layer': 4,
    'xa_grad_reduce':-0.01,

    #dataset
    'batch_size':128,
    #SAL
    'deltaall':20,
    'alpha' :0.5,

    'theta_epoch':10,
    'attack_epoch':2,

    'min_weight':1e8,

    #network
    'network_lr':0.01,
    'num_epochs':1000,

    #full training set though network no SAL
    'FT_epoch':4,

    #partial training starts at and num of batch used
    'partial_epoch_start':4,
    'partial_batch':5,
    'partial_epoch_every':5,
    'partial_epoch_save':50,


    'early_stop':500,

    #for adv data store
    'current_epoch':0,
    'current_batch':0,
    'current_theta_epoch':0,
    'current_attack_epoch':0,

}

In [3]:
import os
from datetime import datetime
from pathlib import Path



def create_folder_based_on_time(base_path: os.PathLike):
    base_path = Path(base_path)
    # Get the current time
    current_time = datetime.now()

    # Format the time as a string with only date, hour, and minute (e.g., "2024-01-23_12-30")
    time_str = current_time.strftime("%m-%d-%H-%M")

    # Create a folder name based on the formatted time
    folder_name = base_path / f"{config_dict['y_target']}_{time_str}"


    for subfolder in ('adv', 'models', 'sorted'):
        os.makedirs(folder_name / subfolder, exist_ok=True)

    return folder_name

In [4]:
created_folder = create_folder_based_on_time(Path.cwd() / config_dict['exp-base-dir'])
config_dict['exp-dir'] = created_folder
created_folder

PosixPath('/home/nicholas/programs/ood-materials/exps/delta_e_02-03-22-34')

In [5]:
# # Step 1: Load the CSV file
# inputdataset =pd.read_feather('./nom_dataset_pred.feather')
# #excluded = ["comp",'delta_e', 'bandgap']
# feature_used_columns = inputdataset.columns[:inputdataset.columns.get_loc('TSNE_x')]
# lables = inputdataset.columns[inputdataset.columns.get_loc('Xshift_tsne'):]

# config_dict['dim_x']=len(inputdataset.columns[1:inputdataset.columns.get_loc('bandgap')])

In [6]:
# Xshift_tsne     =inputdataset.loc[inputdataset['Xshift_tsne'    ]==1][feature_used_columns]
# Xshift_umap     =inputdataset.loc[inputdataset['Xshift_umap'    ]==1][feature_used_columns]
# statY_delta_e   =inputdataset.loc[inputdataset['statY_delta_e'  ]==1][feature_used_columns]
# infoY_delta_e   =inputdataset.loc[inputdataset['infoY_delta_e'  ]==1][feature_used_columns]
# statY_bandgap   =inputdataset.loc[inputdataset['statY_bandgap'  ]==1][feature_used_columns]
# infoY_bandgap   =inputdataset.loc[inputdataset['infoY_bandgap'  ]==1][feature_used_columns]
# inRand1         =inputdataset.loc[inputdataset['inRand1'        ]==1][feature_used_columns]
# inRand2         =inputdataset.loc[inputdataset['inRand2'        ]==1][feature_used_columns]
# inRand3         =inputdataset.loc[inputdataset['inRand3'        ]==1][feature_used_columns]
# inRand4         =inputdataset.loc[inputdataset['inRand4'        ]==1][feature_used_columns]
# inRand5         =inputdataset.loc[inputdataset['inRand5'        ]==1][feature_used_columns]
# inPizoe         =inputdataset.loc[inputdataset['inPizoe'        ]==1][feature_used_columns]


# training_set = inputdataset[(inputdataset[lables] == 0).all(axis=1)][feature_used_columns]


# # Step 3: Define a custom PyTorch Dataset class
# class MyDataset(Dataset):
#     def __init__(self, df):
#         self.inputs = df.drop(columns=['delta_e','comp','bandgap']).values
#         self.labels = df['delta_e'].values

#     def __len__(self):
#         return len(self.inputs)

#     def __getitem__(self, index):
#         input = self.inputs[index].tolist()[:]
#         label = self.labels[index].tolist()
#         return torch.tensor(input, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)
#     def getSALdata(self):
#         input = np.array(self.inputs[:].tolist())
#         label = np.array(self.labels[:].tolist())
#         return (input, label)

# class RecurrentDataset(Dataset):
#     def __init__(self, df):
#         self.inputs = df['data'].values
#         self.labels = df['label'].values

#     def __len__(self):
#         return len(self.inputs)

#     def __getitem__(self, index):
#         input = self.inputs[index].tolist()[:]
#         label = self.labels[index].tolist()
#         return torch.tensor(input, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)


# # Step 4: Use DataLoader to create batches
# batch_size = config_dict['batch_size']

# train_dataset = MyDataset(training_set)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)


# Xshift_tsne_loader      =DataLoader(MyDataset(Xshift_tsne  ),batch_size=len(Xshift_tsne  ))
# Xshift_umap_loader      =DataLoader(MyDataset(Xshift_umap  ),batch_size=len(Xshift_umap  ))
# statY_delta_e_loader    =DataLoader(MyDataset(statY_delta_e),batch_size=len(statY_delta_e))
# infoY_delta_e_loader    =DataLoader(MyDataset(infoY_delta_e),batch_size=len(infoY_delta_e))
# statY_bandgap_loader    =DataLoader(MyDataset(statY_bandgap),batch_size=len(statY_bandgap))
# infoY_bandgap_loader    =DataLoader(MyDataset(infoY_bandgap),batch_size=len(infoY_bandgap))
# Rsplt_testset1_loader   =DataLoader(MyDataset(inRand1      ),batch_size=len(inRand1      ))
# Rsplt_testset2_loader   =DataLoader(MyDataset(inRand2      ),batch_size=len(inRand2      ))
# Rsplt_testset3_loader   =DataLoader(MyDataset(inRand3      ),batch_size=len(inRand3      ))
# Rsplt_testset4_loader   =DataLoader(MyDataset(inRand4      ),batch_size=len(inRand4      ))
# Rsplt_testset5_loader   =DataLoader(MyDataset(inRand5      ),batch_size=len(inRand5      ))
# piezo_test_loader       =DataLoader(MyDataset(inPizoe      ),batch_size=len(inPizoe      ))



# data=train_dataset.getSALdata()


In [7]:
# Step 1: Load the CSV file

Rsplt_testset =pd.read_csv('./dataset/Rsplt_testset.csv', index_col=None)
Xshft_testset =pd.read_csv('./dataset/Xshft_testset.csv', index_col=None)
piezo_testset =pd.read_csv('./dataset/piezo_testset.csv', index_col=None)
statY_testset =pd.read_csv('./dataset/statY_testset.csv', index_col=None)
infoY_testset =pd.read_csv('./dataset/infoY_testset.csv', index_col=None)
final_train   =pd.read_csv('./dataset/final_trainset.csv',index_col=None)

Rsplt_testset1 =pd.read_csv('./dataset/Rsplt_testset1.csv', index_col=None)
Rsplt_testset2 =pd.read_csv('./dataset/Rsplt_testset2.csv', index_col=None)
Rsplt_testset3 =pd.read_csv('./dataset/Rsplt_testset3.csv', index_col=None)
Rsplt_testset4 =pd.read_csv('./dataset/Rsplt_testset4.csv', index_col=None)
Rsplt_testset5 =pd.read_csv('./dataset/Rsplt_testset5.csv', index_col=None)



# Step 3: Define a custom PyTorch Dataset class
class MyDataset(Dataset):
    def __init__(self, df):
        self.inputs = df.drop(columns=['delta_e','pretty_comp']).values
        self.labels = df['delta_e'].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        input = self.inputs[index].tolist()[:]
        label = self.labels[index].tolist()
        return torch.tensor(input, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)
    def getSALdata(self):
        input = np.array(self.inputs[:].tolist())
        label = np.array(self.labels[:].tolist())
        return (input, label)

class RecurrentDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['data'].values
        self.labels = df['label'].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        input = self.inputs[index].tolist()[:]
        label = self.labels[index].tolist()
        return torch.tensor(input, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)


# Step 4: Use DataLoader to create batches
batch_size = config_dict['batch_size']

train_dataset = MyDataset(final_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)


Rsplt_test_dataset = MyDataset(Rsplt_testset)
Xshft_test_dataset = MyDataset(Xshft_testset)
piezo_test_dataset = MyDataset(piezo_testset)
statY_test_dataset = MyDataset(statY_testset)
infoY_test_dataset = MyDataset(infoY_testset)

Rsplt_testset1_dataset = MyDataset(Rsplt_testset1)
Rsplt_testset2_dataset = MyDataset(Rsplt_testset2)
Rsplt_testset3_dataset = MyDataset(Rsplt_testset3)
Rsplt_testset4_dataset = MyDataset(Rsplt_testset4)
Rsplt_testset5_dataset = MyDataset(Rsplt_testset5)

Rsplt_testset1_loader = DataLoader(Rsplt_testset1_dataset, batch_size=len(Rsplt_testset1))
Rsplt_testset2_loader = DataLoader(Rsplt_testset2_dataset, batch_size=len(Rsplt_testset2))
Rsplt_testset3_loader = DataLoader(Rsplt_testset3_dataset, batch_size=len(Rsplt_testset3))
Rsplt_testset4_loader = DataLoader(Rsplt_testset4_dataset, batch_size=len(Rsplt_testset4))
Rsplt_testset5_loader = DataLoader(Rsplt_testset5_dataset, batch_size=len(Rsplt_testset5))

Rsplt_test_loader = DataLoader(Rsplt_test_dataset, batch_size=len(Rsplt_testset))
Xshft_test_loader = DataLoader(Xshft_test_dataset, batch_size=len(Xshft_testset))
piezo_test_loader = DataLoader(piezo_test_dataset, batch_size=len(piezo_testset))
statY_test_loader = DataLoader(statY_test_dataset, batch_size=len(statY_testset))
infoY_test_loader = DataLoader(infoY_test_dataset, batch_size=len(infoY_testset))


data=train_dataset.getSALdata()

In [8]:
def sample_and_remove_from_testset(test_dataloader, num_samples=config_dict['batch_size']):
    # Step 1: Get the entire test set
    test_dataset = test_dataloader.dataset
    seed=69
    torch.manual_seed(seed)

    # Step 2: Sample specified number of indices from the test set
    sampled_indices = torch.randperm(len(test_dataset))

    # Set seed again to ensure consistency in the order of indices between runs
    torch.manual_seed(seed)
    # Select the first num_samples indices from the shuffled indices
    sampled_indices = sampled_indices[:num_samples]

    # Step 3: Create a DataLoader with the sampled test batch
    sampled_test_dataset = Subset(test_dataset, sampled_indices)
    sampled_test_dataloader = DataLoader(sampled_test_dataset, batch_size=num_samples, shuffle=True)

    # Step 4: Create a DataLoader with the remaining test set (excluding the sampled indices)
    remaining_indices = [i for i in range(len(test_dataset)) if i not in sampled_indices]
    remaining_test_dataset = Subset(test_dataset, remaining_indices)
    remaining_test_dataloader = DataLoader(remaining_test_dataset, batch_size=len(remaining_test_dataset), shuffle=True)

    # Step 5: Print the lengths of the original test set, sampled test set, and remaining test set
    print(f"Original Test Set Size: {len(test_dataset)}")
    print(f"Sampled Test Set Size: {len(sampled_test_dataloader.dataset)}")
    print(f"Remaining Test Set Size: {len(remaining_test_dataloader.dataset)}")

    return sampled_test_dataloader, remaining_test_dataloader

In [9]:
piezo_adv_loader,piezo1_test_loader = sample_and_remove_from_testset(piezo_test_loader)

Original Test Set Size: 382
Sampled Test Set Size: 128
Remaining Test Set Size: 254


In [10]:
with open('feat_col_name.txt', 'r') as file:
   column_names = file.read().split()
# column_names = feature_used_columns
def save_tensor(ori_data,ori_lab,adv_data,adv_lab,epoch,batch_idx):
    ori_data_list  = ori_data.clone().cpu().numpy()
    ori_labels_list = ori_lab.clone().cpu().numpy()
    adv_data_list  = adv_data.clone().cpu().numpy()
    adv_labels_list = adv_lab.clone().cpu().tolist()


    csv_filename = config_dict["exp-dir"] / 'adv' / f'adv_data_labels_in_attack{epoch}_{batch_idx}.csv'

    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Write header
        header = column_names+ ['label']#[f'feature_{i}' for i in range(ori_data_list.shape[1])]
        writer.writerow(header)

        # Write data and labels for each group
        rows_group1 = np.column_stack((ori_data_list, ori_labels_list))
        rows_group2 = np.column_stack((adv_data_list, ori_labels_list))


        # Combine data from the first two groups only
        all_rows = np.vstack((rows_group1, rows_group2))

        writer.writerows(all_rows)


In [11]:
class IRNet_intorch(torch.nn.Module):
    #'128-64-16'
    def __init__(self, input_size):
        super(IRNet_intorch, self).__init__()
        self.fc128 =nn.Linear(128, 128)
        self.fc64 =nn.Linear(64, 64)
        self.fc16 =nn.Linear(16, 16)

        self.bn128 =nn.BatchNorm1d(128)
        self.bn64 =nn.BatchNorm1d(64)
        self.bn16 =nn.BatchNorm1d(16)

        self.relu = nn.ReLU()
        self.inputlayer = nn.Linear(input_size, 128)

        self.con128_64 = nn.Linear(128, 64)
        self.con64_16 = nn.Linear(64,16)
        self.output16 = nn.Linear(16,1)




    def forward(self, x):
        x = self.inputlayer(x)

        x_res = x
        x = self.fc128(x)
        x = self.bn128(x)
        x = self.relu(x)
        x = x+x_res
        x = self.con128_64(x)

        x_res = x
        x = self.fc64(x)
        x = self.bn64(x)
        x = self.relu(x)
        x = x+x_res
        x = self.con64_16(x)

        x_res = x
        x = self.fc16(x)
        x = self.bn16(x)
        x = self.relu(x)
        x = x+x_res

        x = self.output16(x)
        return x

In [12]:
train_best_loss      = float('inf')
partial_train_best_loss= float('inf')

Rsplt_ave_best_loss = float('inf')

#Xshift_tsne_best_loss = float('inf')
#Xshift_umap_best_loss = float('inf')
#statY_delta_e_best_loss = float('inf')
#infoY_delta_e_best_loss = float('inf')
#statY_bandgap_best_loss = float('inf')
#infoY_bandgap_best_loss = float('inf')
#Rsplt_testset1_best_loss = float('inf')
#Rsplt_testset2_best_loss = float('inf')
#Rsplt_testset3_best_loss = float('inf')
#Rsplt_testset4_best_loss = float('inf')
#Rsplt_testset5_best_loss = float('inf')
#piezo_test_best_loss = float('inf')

loss_df = pd.DataFrame(columns=[
                        'epoch',
                        'train',
                        'partialtrain',
                        'Xshift_tsne'  ,
                        'Xshift_umap'  ,
                        'statY_delta_e',
                        'infoY_delta_e',
                        'statY_bandgap',
                        'infoY_bandgap',
                        'inRand1'      ,
                        'inRand2'      ,
                        'inRand3'      ,
                        'inRand4'      ,
                        'inRand5'      ,
                        'inPizoe'      ,
                        'RspltAVE',
                        'save',
                        'attack_gamma'
                        ])

In [13]:
def pretty(vector):
    if type(vector) is list:
        vlist = vector
    elif type(vector) is np.ndarray:
        vlist = vector.reshape(-1).tolist()
    else:
        vlist = vector.view(-1).tolist()

    return "[" + ", ".join("{:+.4f}".format(vi) for vi in vlist) + "]"

# Optimizer Class to maximize loss of adversarial dataset
class Adam:
    def __init__(self, learning_rate=1e-3, beta1=0.9, beta2=0.9, epsilon=1e-8):
        self.device = torch.device(device)
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None
        self.v = None
        self.m_hat = None
        self.v_hat = None
        self.initialize = False

    # Grad = adv_gradient
    # Iternum = iteration
    # theta = adv_images
    # Gradient ascent
    def update(self, grad, iternum, theta):
        if not self.initialize:
            self.m = (1 - self.beta1) * grad
            self.v = (1 - self.beta2) * grad ** 2
            self.initialize = True
        else:
            assert self.m.shape == grad.shape
            self.m = self.beta1 * self.m + (1 - self.beta1) * grad
            self.v = self.beta2 * self.v + (1 - self.beta2) * grad ** 2

        self.m_hat = self.m / (1 - self.beta1 ** iternum)
        self.v_hat = self.v / (1 - self.beta2 ** iternum)
        return theta + self.lr * self.m_hat / (self.epsilon + torch.sqrt(self.v_hat))

# Save the configuration to a YAML file
def save_config(config, file_path):
    with Path(file_path).open('w') as file:
        yaml.dump(config, file, default_flow_style=False)

# Load the configuration from the YAML file
def load_config(file_path):
    with Path(file_path).open('r') as file:
        config = yaml.safe_load(file)
    return config


In [14]:
save_config(config_dict, config_dict["exp-dir"] / 'config.yaml')
loaded_config = load_config(config_dict["exp-dir"] / 'config.yaml')
print("Loaded Configuration:", loaded_config)

ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/object/apply:pathlib.PosixPath'
  in "/home/nicholas/programs/ood-materials/exps/delta_e_02-03-22-34/config.yaml", line 16, column 10

In [ ]:
train_dataset.getSALdata()[0].shape

(3697, 150)

In [ ]:
class StableAL:
    def __init__(self, environment):
        self.weights = None
        self.model = None
        self.weight_grad = None
        self.xa_grad = None
        self.theta_grad = None
        self.gamma = None
        self.adversarial_data = None
        self.loss_criterion = torch.nn.MSELoss()

        self.adv_based_on = None
        self.adv_again = None

        self.X = None
        self.y = None

        # init
        # Number of covariates
        dim_x = 150

        self.model = IRNet_intorch(dim_x).to(device)
        # Covariate Weights
        self.weights = torch.zeros(dim_x).reshape(-1, 1) + 100.0
        self.weights = self.weights.to(device)

    def cost_function(self, x, x_adv):
        # Variable cost level where the weights determine the cost level
        cost = torch.mean(((x - x_adv) ** 2).mm(self.weights)).to(device)
        return cost

    # Loss across Training environments
    # Self.loss_criterion = MSELoss
    def r(self, environments, alpha=config_dict['alpha_for_r']):
        result = 0.0
        env_loss = []
        for x_e, y_e in environments:
            x_e = x_e.to(device)
            y_e = y_e.to(torch.float32).to(device)
            env_loss.append(self.loss_criterion(self.model(x_e), y_e))
        env_loss = torch.Tensor(env_loss)
        max_index = torch.argmax(env_loss)
        min_index = torch.argmin(env_loss)

        for idx, (x_e, y_e) in enumerate(environments):
            x_e = x_e.to(device)
            y_e = y_e.to(torch.float32).to(device)
            if idx == max_index:
                result += (alpha + 1) * self.loss_criterion(self.model(x_e), y_e)
            elif idx == min_index:
                result += (1 - alpha) * self.loss_criterion(self.model(x_e), y_e)
            else:
                result += self.loss_criterion(self.model(x_e), y_e)
        return result

    # generate adversarial data
    # Maximize the loss using their own ADAM.update method(their own optimizer)
    def attack(self, gamma, data, step, epoch, batch_idx):
        attack_lr = config_dict['attack_lr']
        images, labels = data
        images_adv = images.clone().detach()

        optimizer = Adam(learning_rate=attack_lr)

        for i in range(step):
            if images_adv.grad is not None:
                images_adv.grad.data.zero_()

            images_adv = images_adv.to(device)
            images_adv.requires_grad_(True)
            outputs = self.model(images_adv)

            labels = labels.float().to(device)
            images = images.to(device)
            loss = self.loss_criterion(outputs, labels) - gamma * self.cost_function(
                images, images_adv
            )

            loss.backward()

            images_adv.data = optimizer.update(images_adv.grad, i + 1, images_adv)

        self.weight_grad = -2 * gamma * attack_lr * (images_adv - images)
        temp_image = images_adv.clone().detach()
        temp_label = labels.clone().detach()
        self.adversarial_data = (temp_image, temp_label)

        # save adv and ori data
        save_tensor(images, labels, temp_image, temp_label, epoch, batch_idx)

        return images_adv, labels

    # Optimizes the model paremeters such that the loss is minimized
    # on the adversarial data from self.attack
    def train_theta(
        self, data, epochs_theta, epoch_attack, gamma, end_flag=False, epoch=0, batch_idx=None
    ):
        optimizer = optim.Adam(self.model.parameters(), lr=config_dict['opt_lr'])
        self.adv_based_on = data
        # For __ Theta epochs_theta
        for i_theta in range(epochs_theta):
            if i_theta % config_dict['advadv_rate'] == 0 or not end_flag:
                images_adv, labels = self.attack(
                    gamma, data, step=epoch_attack, epoch=epoch, batch_idx=batch_idx
                )

            else:
                self.adv_again = self.adversarial_data
                images_adv, labels = self.attack(
                    gamma,
                    self.adversarial_data,
                    step=epoch_attack,
                    epoch=epoch,
                    batch_idx=batch_idx,
                )

            # print(f"original data: {data[0].shape}")
            # print(f"attack data: {images_adv.shape}")
            optimizer.zero_grad()
            images_adv = images_adv.to(device)
            outputs = self.model(images_adv)
            loss = self.loss_criterion(outputs, labels.float())

            if self.xa_grad is None:
                dtheta_dx = []
                dloss_dtheta = grad(loss, self.model.parameters(), create_graph=True)[
                    config_dict['grad_layer']
                ].reshape(-1)

                # size dloss = model.para size
                for name1, param in self.model.named_parameters():
                    print(f'grad: {name1}          {param.shape}')

                # time.sleep(5.5)    # Pause 5.5 seconds
                print(f'dloss_dtheta.shape:  {dloss_dtheta.shape[0]}')
                for j in range(dloss_dtheta.shape[0]):
                    # print(f"dloss_dtheta.shape[0]:j     {j}")
                    dtheta_dx.append(
                        grad(dloss_dtheta[j], images_adv, create_graph=True)[0].detach()
                    )

                self.xa_grad = torch.stack(dtheta_dx, 1).detach()

            else:
                dloss_dtheta = grad(loss, self.model.parameters(), create_graph=True)[
                    config_dict['grad_layer']
                ].reshape(-1)
                dtheta_dx = []

                for j in range(dloss_dtheta.shape[0]):
                    dtheta_dx.append(
                        grad(dloss_dtheta[j], images_adv, create_graph=True)[0].detach()
                    )
                self.xa_grad += torch.stack(dtheta_dx, 1).detach()

            # print(f"xa_grad size: {self.xa_grad.shape}")
            del dtheta_dx
            del dloss_dtheta
            torch.cuda.empty_cache()

            # print('%d | %.4f | %s'%(i, loss, pretty(self.model.layer[4].weight)))
            # if i % 1000 == 999:

            # print(f"loss?")
            loss.backward(retain_graph=True)
            # print(f"step?")
            optimizer.step()
            # print(f"step!")
        self.xa_grad *= config_dict['xa_grad_reduce']

In [ ]:
for param in method.model.parameters():
    print(param.shape)

torch.Size([128, 128])
torch.Size([128])
torch.Size([64, 64])
torch.Size([64])
torch.Size([16, 16])
torch.Size([16])
torch.Size([128])
torch.Size([128])
torch.Size([64])
torch.Size([64])
torch.Size([16])
torch.Size([16])
torch.Size([128, 150])
torch.Size([128])
torch.Size([64, 128])
torch.Size([64])
torch.Size([16, 64])
torch.Size([16])
torch.Size([1, 16])
torch.Size([1])


In [ ]:
method = StableAL([train_dataset.getSALdata()])

In [ ]:
adv_data,adv_target= next(iter(piezo_adv_loader))

In [ ]:
import warnings
import csv

warnings.filterwarnings("ignore")

min_weight = torch.min(method.weights)
attack_gamma = (1.0 / min_weight).data
criterion = nn.MSELoss()
epoch = 0
zero_list = []
end_flag = False

method.model =IRNet_intorch(150).to(device)
method.model.optimizer = optim.Adam(method.model.parameters(), lr=config_dict['network_lr'])
# Create a dictionary to store the best loss for each dataset
best_loss_dict = {}
all_loss_dict = {}

while epoch <=config_dict['num_epochs']:
    train_loss = 0.0

    #Xshift_tsne_mse_loss = 0
    #Xshift_umap_mse_loss = 0
    #statY_delta_e_mse_loss = 0
    #infoY_delta_e_mse_loss = 0
    #statY_bandgap_mse_loss = 0
    #infoY_bandgap_mse_loss = 0
    #Rsplt_testset1_mse_loss = 0
    #Rsplt_testset2_mse_loss = 0
    #Rsplt_testset3_mse_loss = 0
    #Rsplt_testset4_mse_loss = 0
    #Rsplt_testset5_mse_loss = 0
    #piezo_test_mse_loss = 0

    partial_train_loss = 0.0
    total_train_loss = 0.0

    minima = []
    for batch_idx, (data, target) in enumerate(train_loader):
        print(f"current in epoch    {epoch}      batch {batch_idx}")

         # Zero the gradients
        method.model.optimizer.zero_grad()

        # Forward pass
        outputs = method.model(data.to(device))
        loss = criterion(outputs.to(device) , target.float().to(device))

        # Backward pass and optimization
        loss.backward()
        method.model.optimizer.step()
        partial_train_loss += loss.cpu().item()
        if epoch <config_dict['FT_epoch']:
            continue

        method.train_theta((adv_data, adv_target), config_dict['theta_epoch'], config_dict['attack_epoch'], attack_gamma, end_flag, epoch, batch_idx)
        rtheta = method.r([[adv_data, adv_target]], alpha=config_dict['alpha'] / math.sqrt(epoch + 1))
        method.theta_grad = grad(rtheta, list(method.model.parameters()), create_graph=True, allow_unused=True)
        dr_dx = torch.matmul(method.theta_grad[config_dict['grad_layer']].reshape(-1), method.xa_grad).squeeze()
        deltaw = dr_dx * method.weight_grad
        deltaw = torch.sum(deltaw, 0)

        deltaw[zero_list] = 0.0
        max_grad = torch.max(torch.abs(deltaw))
        deltastep = config_dict['deltaall']
        lr_weight = (deltastep / max_grad).detach()
        print(f'RLoss: {rtheta.data}')

        if epoch >config_dict['partial_epoch_start']:

            if batch_idx ==config_dict['partial_batch']:# train partial trainset
                break



    partial_train_loss /= (batch_idx+1)
    print("=="*20)
    print(f"Epoch {epoch+1}/{config_dict['num_epochs']} - partial_train_loss: {partial_train_loss:.4f} ")
    if epoch %config_dict['partial_epoch_every']==0:
        with torch.no_grad():
                print("sorting training set")
                # for sorting Training set
                sort_MAE=[]
                method.model.eval()
                for i in range(len(train_dataset.inputs)):
                    inp = train_dataset.inputs[i]
                    tar = train_dataset.labels[i]

                    x = torch.tensor([inp.tolist()], dtype=torch.float32).to(device)
                    y = torch.tensor(tar.tolist(), dtype=torch.float32).to(device)

                    output = method.model(x)
                    loss = criterion(output, y).cpu()
                    # Accumulate the training loss
                    train_loss += loss.item()
                    #print(f"loss:       {loss}")
                    sort_MAE.append({'data' : inp,
                                                'label' :tar,
                                                'loss' : loss})


                if epoch%config_dict['partial_epoch_save']==0:
                    sort_MAE = pd.DataFrame(sort_MAE)
                    sort_MAE.to_csv(config_dict['exp-dir'] / 'sorted/train_set_sorting_{epoch}.csv',index=False)

                sort_MAE = pd.DataFrame(sort_MAE)
                new_train = sort_MAE.sort_values(by=['loss'],ascending=False)
                new_train_dataset = RecurrentDataset(new_train)
                train_loader = DataLoader(new_train_dataset, batch_size=batch_size, shuffle=False,drop_last=True)

                train_loss /= len(train_dataset.inputs)
                print(f"Epoch {epoch+1}/{config_dict['num_epochs']} - Training loss: {train_loss:.4f} ")
                print("=="*20)


    mse_losses_dict = {}

    for batch_idx, (data, target) in enumerate(train_loader):

            outputs = method.model(data.to(device))
            loss = criterion(outputs , target.float().to(device))
            train_loss += loss.item()
    train_loss /= len(train_loader)

    print(f'Epoch: [{(epoch + 1)}/{ config_dict["num_epochs"] }], TrainLoss: {train_loss}')

    mse_losses_dict["Train"] = train_loss
    mse_losses_dict["Partial_train"] = partial_train_loss

    def calculate_mse_loss(data_loader, model, criterion, device):
        mse_loss = 0.0
        total_batches = len(data_loader)

        for batch_idx, (data, target) in enumerate(data_loader):
            outputs = model(data.to(device))
            loss = criterion(outputs, target.float().to(device))
            mse_loss += loss.item()

        mse_loss /= total_batches
        return mse_loss


    data_loaders = {
         # 'Xshift_tsne': Xshift_tsne_loader, 'Xshift_umap': Xshift_umap_loader,
                # 'statY_delta_e': statY_delta_e_loader, 'infoY_delta_e': infoY_delta_e_loader,
                # 'statY_bandgap': statY_bandgap_loader, 'infoY_bandgap': infoY_bandgap_loader,
                'Rsplt_testset1': Rsplt_testset1_loader, 'Rsplt_testset2': Rsplt_testset2_loader,
                'Rsplt_testset3': Rsplt_testset3_loader, 'Rsplt_testset4': Rsplt_testset4_loader,
                'Rsplt_testset5': Rsplt_testset5_loader, 'piezo_test': piezo1_test_loader}

    # Iterate over each data loader and calculate MSE loss
    for dataset_name, loader in data_loaders.items():
        loader_mse_loss = calculate_mse_loss(loader, method.model, criterion, device)
        # Append the MSE loss to the list in the dictionary
        mse_losses_dict[dataset_name] = loader_mse_loss


    rsplt_ave = np.average([
                                mse_losses_dict["Rsplt_testset1"],
                                mse_losses_dict["Rsplt_testset2"],
                                mse_losses_dict["Rsplt_testset3"],
                                mse_losses_dict["Rsplt_testset4"],
                                mse_losses_dict["Rsplt_testset5"],

                                ])
    mse_losses_dict["rsplt_ave"] = rsplt_ave

    save =[]
    # Separate loop to update the best loss for all datasets
    for dataset_name, loss in mse_losses_dict.items():
        if dataset_name not in best_loss_dict or loader_mse_loss < best_loss_dict[dataset_name]:
            best_loss_dict[dataset_name] = loader_mse_loss
            save.append(dataset_name)


    # Stop the training process if the training loss has stopped decreasing or has started to increase
    if train_loss < train_best_loss:
        train_best_loss = train_loss
        counter = 0
        torch.save(method.model, config_dict['exp-dir'] / f'models/IR3_epoch_{epoch}.pt')
        torch.save(method.weights, config_dict['exp-dir'] / 'models/SAL_weight_{epoch}_gamma_{attack_gamma}.pt')
        save.append("Train")
    else:
        counter += 1
        print(f'training Loss has not improved for {counter} epochs.')



    if rsplt_ave < Rsplt_ave_best_loss:
            Rsplt_ave_best_loss  =  rsplt_ave
            counter_val = 0
            torch.save(method.model,config_dict['exp-dir'] / "models/IR3_SAL-bset-Rsplt_test_mse_loss.pt")
            save.append("Rsplt_AVE")
    else:
        counter_val += 1
        if counter_val >= config_dict['early_stop']:
            print(f'Training stopped. Valid (rand) Loss has not improved for {500} epochs.')
            break

    all_loss_dict[epoch + 1] = mse_losses_dict

    mse_losses_df = pd.DataFrame.from_dict(all_loss_dict, orient='index')


    mse_losses_df.reset_index().rename(columns={'index': 'epoch'}).to_feather(config_dict['exp-dir']/ 'SAL-training_loss.feather')
    epoch=epoch+1

    # adjust gamma according to min(weight)

    for i in range(method.weights.shape[0]):
        if method.weights[i] > 0.0 and method.weights[i] < min_weight:
            min_weight = method.weights[i]
        if method.weights[i] < 0.0:
            method.weights[i] = 1.0
            zero_list.append(i)

    attack_gamma = (1.0 / min_weight).data
    if epoch <=config_dict['FT_epoch']:
        continue

    method.weights -= lr_weight * deltaw.detach().reshape(method.weights.shape)
    del rtheta
    del dr_dx
    del deltaw
    del max_grad
    del deltastep
    del lr_weight
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()





current in epoch    0      batch 0
current in epoch    0      batch 1
current in epoch    0      batch 2
current in epoch    0      batch 3
current in epoch    0      batch 4
current in epoch    0      batch 5
current in epoch    0      batch 6
current in epoch    0      batch 7
current in epoch    0      batch 8
current in epoch    0      batch 9
current in epoch    0      batch 10
current in epoch    0      batch 11
current in epoch    0      batch 12
current in epoch    0      batch 13
current in epoch    0      batch 14
current in epoch    0      batch 15
current in epoch    0      batch 16
current in epoch    0      batch 17
current in epoch    0      batch 18
current in epoch    0      batch 19
current in epoch    0      batch 20
current in epoch    0      batch 21
current in epoch    0      batch 22
current in epoch    0      batch 23
current in epoch    0      batch 24
current in epoch    0      batch 25
current in epoch    0      batch 26
current in epoch    0      batch 27
Ep

KeyboardInterrupt: 

In [ ]:
torch.save(method.weights,  config_dict['exp-dir'] / f'Whole_SAL_{epoch}.pt')

In [ ]:
torch.save(method.model,  config_dict['exp-dir'] / f'IR3_epoch_{epoch}.pt')